In [1]:
import os
import requests
import pandas as pd
import spotipy
import spotipy.util as util
import json
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
from time import sleep

In [2]:
# Spotify API Keys
import config
os.environ['SPOTIPY_CLIENT_ID']=config.spotify_cliend_id
os.environ['SPOTIPY_CLIENT_SECRET']=config.spotify_client_secret

In [3]:
# Generate API Token
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = client_credentials_manager.get_access_token()
# print(token)

In [4]:
# Load clean combined weekly data
WORST_SONG_FILE_NAME = os.path.join('data', 'us', '2018-worst-songs.csv')
worst_song_df = pd.read_csv(WORST_SONG_FILE_NAME)
worst_song_df.head()


,Position,Artist,Title
0,1,Post Malone,Psycho (feat. Ty Dolla $ign)
1,2,6ix9ine,FEFE (feat. Nicki Minaj & Murda Beatz)
2,3,Bella Thorne,B*TCH I’M BELLA THORNE
3,4,Justin Timberlake,The Hard Stuff
4,5,Maroon 5,Girls Like You (feat. Cardi B)


In [5]:
features_list = []
for position in worst_song_df['Position']:
    title = worst_song_df.loc[worst_song_df['Position']==position, 'Title'].item()
    artist = worst_song_df.loc[worst_song_df['Position']==position, 'Artist'].item()
    results = sp.search(q=f'track:{title} artist:{artist}', type='track', limit=1)
    track_id = results['tracks']['items'][0]['id']
    print(f'{position} - {track_id}')
    feature_url = 'https://api.spotify.com/v1/audio-features/{0}'.format(track_id)
    feature_rsp = requests.get(feature_url, headers={'Authorization': 'Bearer '+token})
    features = feature_rsp.json()
    
    track_rsp = requests.get(features['track_href'], headers={'Authorization': 'Bearer '+token})
    track_info = track_rsp.json()
    
    features_as_dict = {
        'Position':position,
        'URL': track_info['external_urls']['spotify'],
        'Acousticness': features['acousticness'],
        'Danceability': features['danceability'],
        'Duration (ms)': features['duration_ms'],
        'Energy': features['energy'],
        'Instrumentalness': features['instrumentalness'],
        'Key': features['key'],
        'Liveness': features['liveness'],
        'Loudness': features['loudness'],
        'Mode': features['mode'],
        'Speechiness': features['speechiness'],
        'Tempo': features['tempo'],
        'Time Signature': features['time_signature'],
        'Valence': features['valence'],
        'Album': track_info['album']['name'],
        'Image': track_info['album']['images'][1]['url'],
        'Explicit': track_info['explicit'],
        'Popularity': track_info['popularity']
    }
    features_list.append(features_as_dict)
    sleep(1) # Throttle API calls
 
features_df = pd.DataFrame(features_list)
enriched_df = pd.merge(worst_song_df, features_df, how='left', on='Position')
enriched_df.head()

1 - 3swc6WTsr7rl9DqQKQA55C
2 - 0wcNwiRtEvMNxcUc9pyuuh
3 - 5HhFQfdaPyiC4mjt2TFxxP
4 - 5ltBz3a5wh9IWwdhBaHu4j
5 - 6FRLCMO5TUHTexlWo8ym1W
6 - 0aVsVsOYDSEEigiwTrIab9
7 - 4w8niZpiMy6qz1mntFA5uM
8 - 4S8d14HvHb70ImctNgVzQQ
9 - 1OWSsdbAXFyPDo6khogqAW
10 - 58QhkbaAkLFnn7JwAnAato


,Position,Artist,Title,Acousticness,Album,Danceability,Duration (ms),Energy,Explicit,Image,...,Key,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,Time Signature,URL,Valence
0,1,Post Malone,Psycho (feat. Ty Dolla $ign),0.5800,beerbongs & bentleys,0.739,221440,0.559,True,https://i.scdn.co/image/77eb7c17cafe5502589602...,...,8,0.1120,-8.011,1,88,0.1170,140.124,4,https://open.spotify.com/track/3swc6WTsr7rl9Dq...,0.439
1,2,6ix9ine,FEFE (feat. Nicki Minaj & Murda Beatz),0.0908,Hottest Hip Hop 2018,0.930,179400,0.386,True,https://i.scdn.co/image/dc8f91ccdfc348c77922ce...,...,1,0.1410,-9.130,1,42,0.4040,125.998,4,https://open.spotify.com/track/0wcNwiRtEvMNxcU...,0.383
2,3,Bella Thorne,B*TCH I’M BELLA THORNE,0.2530,B*TCH I'M BELLA THORNE,0.639,194322,0.754,True,https://i.scdn.co/image/ca99554561e0a00c4b63ba...,...,8,0.0904,-4.137,1,46,0.1790,145.036,4,https://open.spotify.com/track/5HhFQfdaPyiC4mj...,0.743
3,4,Justin Timberlake,The Hard Stuff,0.1520,Man of the Woods,0.501,195387,0.839,False,https://i.scdn.co/image/9f4f07d164fa636b6ebc37...,...,11,0.3670,-7.472,1,50,0.0432,167.853,4,https://open.spotify.com/track/5ltBz3a5wh9IWwd...,0.489
4,5,Maroon 5,Girls Like You (feat. Cardi B),0.5680,Girls Like You (feat. Cardi B),0.851,235545,0.541,True,https://i.scdn.co/image/b413991f48b307c7e5b43d...,...,0,0.1300,-6.825,1,89,0.0505,124.959,4,https://open.spotify.com/track/6FRLCMO5TUHTexl...,0.448


In [6]:
ENRICHED_FILE_NAME = os.path.join('data', 'clean', '2018-worst-songs-enriched.csv')

if not os.path.exists(os.path.dirname(ENRICHED_FILE_NAME)):
    os.mkdir(os.path.dirname(ENRICHED_FILE_NAME))
    
enriched_df.to_csv(ENRICHED_FILE_NAME, index=False)